### An efficient autocomplete solution that allows us to add weights to each inserted word

This will follow a trie structure. The idea is that each character is a node that maps to a dicitonary of only the subsequent characters in the inserted words. Each node also has attribute endtoken, telling us to stop since we've found the end of a word and also has a weight parameter

In [1]:
class TrieNode(object):
    def __init__(self):
        
        self.children = {}
        self.endtoken = False
        self.weight = None

class Trie(object):
    def __init__(self):
        self.trie = TrieNode()
        
    def insert(self,word,w):
        
        c_node = self.trie
        for char in word:
            if char not in c_node.children:
                c_node.children[char] = TrieNode()
            c_node = c_node.children[char]
        
        c_node.endtoken = True
        c_node.weight = w
            
        
    def find(self,prefix):
        
        #First, iterate as far as we can go in the prefix. Then, we need to 
        #recurse though all the possible suffixes and surface them
        c_node = self.trie
        for char in prefix:
            if char in c_node.children:
                c_node = c_node.children[char]
            else:
                return []
        
        return self._generate_suggestions(c_node)
        
    def _generate_suggestions(self,node):
        
        result = []

        for char, suffix_node in node.children.items():
            
            #End condition for the recursion. Its easy just to add the weight as a string so
            #we don't have to specify a special case
            if suffix_node.endtoken == True:
                subresult = [char+"#%s"%suffix_node.weight]
            #Otherwise, add chars to the recursion
            else:
                s = self._generate_suggestions(suffix_node)
                subresult = [char + s for s in self._generate_suggestions(suffix_node)]
            
            result.extend(subresult)
        
        return result

In [2]:
autocomplete_trie = Trie()

In [3]:
words = ["rainbow","rainforest","raincatcher","ratty","realistic","really","rank"]

In [4]:
for w in words:
    autocomplete_trie.insert(w,len(w))

In [5]:
def autocomplete(prefix):
        
    autocomplete_strings = [prefix + x for x in autocomplete_trie.find(prefix)]
    autocomplete_dict = {k.split("#")[0]:float(k.split("#")[1]) for k in autocomplete_strings}
    
    return autocomplete_dict

In [6]:
autocomplete("rai")

{'rainbow': 7.0, 'rainforest': 10.0, 'raincatcher': 11.0}